In [7]:
from ghost import Ghost
ghost = Ghost()

with ghost.start() as session:
    page, resource = session.open('http://www.edmunds.com/mazda/cx-5/2016/suv/')

In [ ]:
import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import *  

#Take this class for granted.Just use result of rendering.
class Render(QWebPage):  
    def __init__(self, url):  
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.mainFrame().load(QUrl(url))  
        self.app.exec_()
  
    def _loadFinished(self, result):  
        self.frame = self.mainFrame()  
        self.app.quit()  

url = 'http://www.edmunds.com/mazda/cx-5/2016/suv/' 
r = Render(url)  
result = r.frame.toHtml()
#This step is important.Converting QString to Ascii for lxml to process
print str(result.toAscii())

In [1]:
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [24]:
"""
get the price for each kind of car from edmunds.com

1. get car list
2. for each car, request the corresponding url and crawl the price using beautiful soup
"""

# 1. Be careful not to overwrite saved data, always use 'r'
import pickle
with open('saved-data/python-data/car_list_url.pickle', 'rb') as f:
    car_url = pickle.load(f)
with open('saved-data/python-data/car_list.pickle', 'rb') as f:
    car_list = pickle.load(f)

print(car_list[0])
print(car_url[0])

{'make': 'acura', 'type': 'sedan', 'model': 'ilx', 'year': '2015'}
/acura/ilx/2015/sedan/


In [29]:
res = requests.get('http://www.edmunds.com/inventory/srp.html?radius=1000&zip=27606&make=Acura&model=ILX&year=2015&sub=ILX%20Sedan&trim=
price_soup = BeautifulSoup(res.content, 'lxml')

In [72]:
new_result = []
missing_prices = []

for i in range(len(car_list)):
    car = car_list[i]
    query = 'http://www.edmunds.com/inventory/srp.html?'+'radius=100000&zip=27606' + \
             '&make=' + car['make'] + '&model=' + car['model'] + \
             '&year=' + car['year'] + '&sub=' + car['model'] + '%20' + car['type'] + '&trim='
        
    print 'query: ', query
    res = requests.get(query)
    if res.ok == True:
        price_soup = BeautifulSoup(res.content, 'lxml')
        try:
            price = price_soup.find('span', class_='msrp-price').text
            new_result.append([str(i+1), float(''.join(price.split()[1][1:].split(',')))])
        except:         
            print 'element unavailable'
            missing_prices.append(car)
    else:
        print 'page unavailable'
        missing_prices.append(car)
        
    print
    print

query:  http://www.edmunds.com/inventory/srp.html?radius=100000&zip=27606&make=acura&model=ilx&year=2015&sub=ilx%20sedan&trim=


query:  http://www.edmunds.com/inventory/srp.html?radius=100000&zip=27606&make=acura&model=ilx&year=2016&sub=ilx%20sedan&trim=


query:  http://www.edmunds.com/inventory/srp.html?radius=100000&zip=27606&make=acura&model=mdx&year=2015&sub=mdx%20suv&trim=


query:  http://www.edmunds.com/inventory/srp.html?radius=100000&zip=27606&make=acura&model=mdx&year=2016&sub=mdx%20suv&trim=


query:  http://www.edmunds.com/inventory/srp.html?radius=100000&zip=27606&make=acura&model=rdx&year=2015&sub=rdx%20suv&trim=


query:  http://www.edmunds.com/inventory/srp.html?radius=100000&zip=27606&make=acura&model=rdx&year=2016&sub=rdx%20suv&trim=


query:  http://www.edmunds.com/inventory/srp.html?radius=100000&zip=27606&make=acura&model=rlx&year=2016&sub=rlx%20hybrid&trim=


query:  http://www.edmunds.com/inventory/srp.html?radius=100000&zip=27606&make=acura&model=rlx&year=2015

In [78]:
with open('saved-data/python-data/missing-price.pickle', 'w') as f:
    pickle.dump(missing_prices, f)

In [1]:
import pickle
with open('saved-data/python-data/car-price.pickle', 'r') as f:
    prices = pickle.load(f)

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(prices)

In [17]:
df = df.rename(columns={0: 'CAR_ID', 1: 'PRICE'})

In [20]:
df = df.set_index('CAR_ID')

In [22]:
df.to_csv('saved-data/feature-list/car_price.csv')

In [23]:
len(prices)

936